In [1]:
import pandas as pd

df=pd.read_csv('../data/fraud.csv', index_col = 0)

# Check your data and define X and y : What is X and what is y? 
y = df['Class'].values
df = df.iloc[:,1:]
X = df.drop(columns = 'Class').values
df.shape

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\fraud.csv'

In [2]:
# perform train - test split. (40% as testing and 60% as training)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

#X_train, X_test, y_train, y_test = train_test_split()

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                    test_size = 0.40, 
                                    random_state=1)
X_train.shape

(13015, 29)

In [3]:
def my_KNN_classifier(X_train, y_train):
    #--------------------------------------------------
    ## ----------- K-NN Classifier ------------------## 
    #--------------------------------------------------

    from sklearn.neighbors import KNeighborsClassifier

    steps = [('scaler', StandardScaler()),         
            ('knn', KNeighborsClassifier(n_neighbors = 3))]

    knn_pipeline = Pipeline(steps)
    knn_pipeline.fit(X_train, y_train)

    #--------------------------------------------------
    ## Model Evaluation ## 
    #--------------------------------------------------


    ypred_test = knn_pipeline.predict(X_test)
    report_clf = classification_report(y_test, ypred_test, output_dict= True)

    macro_avg_precision = report_clf['macro avg']['precision']
    macro_avg_recall = report_clf['macro avg']['recall']
    
    return macro_avg_precision, macro_avg_recall

In [4]:
def my_SVC_classifier(X_train, y_train):
    from sklearn.svm import SVC

    ## Linear Kernel  ---------------
    steps = [('scaler', StandardScaler()),         
            ('svc', SVC(kernel = 'linear',
                        class_weight='balanced'))]

    svcL_pipeline = Pipeline(steps)
    svcL_pipeline.fit(X_train, y_train)
    ypred_test = svcL_pipeline.predict(X_test)
    report_clf = classification_report(y_test, ypred_test, output_dict= True)
    
    lr_precision = report_clf['macro avg']['precision']
    lr_recall = report_clf['macro avg']['recall']

    ## Polynomial Kernel -----------------------
    steps = [('scaler', StandardScaler()),         
            ('svc', SVC(kernel = 'poly', degree = 3, 
                        class_weight='balanced'))]

    svcPoly_pipeline = Pipeline(steps)
    svcPoly_pipeline.fit(X_train, y_train)
    ypred_test = svcPoly_pipeline.predict(X_test)
    report_clf = classification_report(y_test, ypred_test, output_dict= True)
    
    poly_precision = report_clf['macro avg']['precision']
    poly_recall = report_clf['macro avg']['recall']

    ## RBF Kernel -----------------------
    steps = [('scaler', StandardScaler()),         
            ('svc', SVC(kernel = 'rbf', gamma = 'scale',
                        class_weight='balanced'))]

    svcRBF_pipeline = Pipeline(steps)
    svcRBF_pipeline.fit(X_train, y_train)
    ypred_test = svcRBF_pipeline.predict(X_test)
    report_clf = classification_report(y_test, ypred_test, output_dict= True)

    rbf_precision = report_clf['macro avg']['precision']
    rbf_recall = report_clf['macro avg']['recall']
    
    return lr_precision, lr_recall, poly_precision, poly_recall, rbf_precision, rbf_recall

In [5]:
df_slr = {}
knn_p, knn_recall = my_KNN_classifier(X_train, y_train)
df_slr['KNN'] = [knn_p, knn_recall]
lr_precision, lr_recall, poly_precision, poly_recall, rbf_precision, rbf_recall = my_SVC_classifier(X_train, y_train)
df_slr['SVC-linear'] = [lr_precision, lr_recall]
df_slr['SVC-Poly'] = [poly_precision, poly_recall]
df_slr['SVC-rbf'] = [rbf_precision, rbf_recall]
df_table = pd.DataFrame.from_dict(df_slr, orient ='index')   
df_table.columns = ['precision', 'recall'] 

c:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [6]:
df_table

,precision,recall
KNN,0.984973,0.897635
SVC-linear,0.665705,0.927439
SVC-Poly,0.828658,0.894532
SVC-rbf,0.767208,0.881534
